# DALL-E playbook

### Mal Minhas, 04.11.22
v0.1

## Introduction

You can now access DALL-E via the OpenAI Images API functionality described [here](https://beta.openai.com/docs/guides/images).

> The Images API provides three methods for interacting with images:
> * Creating images from scratch based on a text prompt
> * Creating edits of an existing image based on a new text prompt
> * Creating variations of an existing image`

## Configuration

In order to use the OpenAI API you will need both an API key and an Organization ID for authentication.  You can set up your API key from your [API Keys page](https://beta.openai.com/account/api-keys).  Organization IDs can be found on your [Organization settings](https://beta.openai.com/account/org-settings) page. The following code assumes your API key and Organization ID have been stored in a local storage in two files called `.openAIKey` and `.openAIOrg`.  We use them to configure two environment variables:

In [1]:
import os

def configureEnvVars(key, org_id):
    # NB: host url is not prepended with \"https\" nor does it have a trailing slash.
    os.environ['OPENAI_API_KEY'] = key
    os.environ['OPENAI_ORG_ID'] = org_id

def getOrganisationId(file):
    with open(file) as f:
        key = f.read()
    return key
    
def getAPIKey(file):
    with open(file) as f:
        key = f.read()
    return key

configureEnvVars(getAPIKey('.openAIKey'),getOrganisationId('.openAIOrg'))

We can now setup OpenAI authentication using these two environment variables.  The code below checks access is working by retreiving a models list:

In [2]:
import os
import openai

openai.organization = os.getenv("OPENAI_ORG_ID")
openai.api_key = os.getenv("OPENAI_API_KEY")
models = openai.Model.list()

Let's enumerate what models are supported:

In [3]:
names = []
for model in models.data:
    names.append(model.get('id'))
print(names)

['curie-similarity', 'text-davinci-insert-001', 'code-search-babbage-text-001', 'babbage-search-query', 'text-davinci-001', 'davinci-search-query', 'text-search-babbage-query-001', 'text-search-curie-query-001', 'davinci-instruct-beta', 'text-curie-001', 'code-search-babbage-code-001', 'text-davinci-edit-001', 'text-search-davinci-doc-001', 'text-search-babbage-doc-001', 'ada', 'davinci', 'text-similarity-babbage-001', 'ada-similarity', 'text-davinci-insert-002', 'text-search-curie-doc-001', 'code-search-ada-code-001', 'babbage', 'text-davinci-002', 'text-ada-001', 'text-similarity-ada-001', 'ada-search-query', 'text-babbage-001', 'curie-instruct-beta', 'babbage-code-search-code', 'code-search-ada-text-001', 'babbage-similarity', 'ada-search-document', 'curie-search-document', 'curie', 'ada-code-search-text', 'text-search-davinci-query-001', 'audio-transcribe-001', 'text-search-ada-doc-001', 'babbage-code-search-text', 'davinci-search-document', 'text-search-ada-query-001', 'text-simil

## Prompt Engineering - text to image

Now you can input your prompt:

In [4]:
prompt_text = input("Please enter your prompt ->")

Please enter your prompt -> a well functioning engineering department


In [5]:
%%time

import requests

def textToImage(text, target, show=False):
    response = openai.Image.create(
      prompt=text,
      n=1,
      size="1024x1024"
    )
    image_url = response['data'][0]['url']
    img_data = requests.get(image_url).content
    with open(target, 'wb') as handler:
        handler.write(img_data)
    return image_url

image_url = textToImage(prompt_text, 'dalle_image.png')

CPU times: user 73.8 ms, sys: 96.1 ms, total: 170 ms
Wall time: 8.61 s


In [6]:
import IPython.display
#IPython.display.Image(url=image_url, width=600, height=450)
IPython.display.Image(url=image_url, width=600, height=450)

## Prompt Engineering - using a seed image

We can now use the image from the previous stage as input into another generation stage to get modified results.  The DALL-E documentation refers to this as an [<em>edit</em>](https://beta.openai.com/docs/guides/images/introduction) of the original image.   In this case we are turning our image into a cartoon through a hint in the prompt text.  You can also create a <em>variation</em> of the original 

In [7]:
%%time

def imageEdit(text, source, target):
    response = openai.Image.create_edit(
      image=open(source, "rb"),
      mask=open("mask.png", "rb"),
      prompt=text,
      n=1,
      size="1024x1024"
    )
    image_url = response['data'][0]['url']
    img_data = requests.get(image_url).content
    with open(target, 'wb') as handler:
        handler.write(img_data)
    return image_url
    
def imageVariation(source, target):
    response = openai.Image.create_variation(
      image=open(source, "rb"),
      n=1,
      size="1024x1024"
    )
    image_url = response['data'][0]['url']
    img_data = requests.get(image_url).content
    with open(target, 'wb') as handler:
        handler.write(img_data)
    return image_url

image_url = imageEdit(prompt_text+' cartoon', 'dalle_image.png', 'dalle_image2.png')

CPU times: user 128 ms, sys: 205 ms, total: 333 ms
Wall time: 30.8 s


In [8]:
import IPython.display
IPython.display.Image(url=image_url, width=600, height=450)

Far too cartoon-like.  Let's try an image variation of our original generated image:

In [9]:
%%time
image_url = imageVariation('dalle_image.png', 'dalle_image3.png')

CPU times: user 91 ms, sys: 115 ms, total: 206 ms
Wall time: 20.1 s


In [10]:
import IPython.display
IPython.display.Image(url=image_url, width=600, height=450)

This is more interesting.